# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [3]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Project 12/Dataset/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall(project_path+"glovefiles")

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [6]:
import pandas as pd

%cd /content/drive/My Drive/Project\ 12/Dataset/
bodies = pd.read_csv("train_bodies.csv")
stances = pd.read_csv("train_stances.csv")

/content/drive/My Drive/Project 12/Dataset


In [7]:
bodies.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...



<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [8]:
dataset = pd.merge(bodies, stances, on='Body ID')
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20


### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
t = Tokenizer(num_words=MAX_NB_WORDS,filters= '!"#$%&()*+,-./:;<=>?@[\]^_`{|}\n“~')

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
t.fit_on_texts(dataset['articleBody'])
t.fit_on_texts(dataset['Headline'])

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
from nltk.tokenize import sent_tokenize,word_tokenize

texts = dataset["articleBody"]

articles = []

for wholeArticle in texts:
  articles.append(sent_tokenize(wholeArticle))

## Check 2:

first element of texts and articles should be as given below. 

In [15]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [16]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import numpy as np

data = np.zeros((len(articles),MAX_SENTS,MAX_SENT_LENGTH),dtype=int)

In [18]:
data.shape


(49972, 20, 20)

In [0]:
i=0
for article in articles:
  j=0
  for sentence in article:
    if j < 20:
      wordArr =  text_to_word_sequence(sentence,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“', lower=True, split=' ')
      k=0
      for word in wordArr:
        if k < 20:
          data[i,j,k] = round(t.word_index[word])
          k+=1
      j+=1
  i+=1



### Check 3:

Accessing first element in data should give something like given below.

In [20]:
data[0, :, :]

array([[    3,   479,   425,  7159,    81,     3,  3721,   334,     5,
         3871,   352,     4,  1430,  2946,     1,    89,    12,   464,
            0,     0],
       [  758,    96,  1046,     3,  2676,  1750,     7,   190,     3,
         1219,  1056,  1967,   701,   160,     1,  3020,   449,     1,
          553,   235],
       [   89,  1067,  4095,  2344,    12,     3,  1092,  3297,    19,
            1,    89,     2,  1789,     1,   520,  2008,    15,     9,
            3,  2912],
       [  186,  3629,   973,   200,  2551,    44,  6727,  1719,  1253,
            5, 13097, 17549,     1,   775,    30,   743,  3975,    67,
           86,     0],
       [ 2344,    12,  1556,    38,  1094,   354,   774,     2,   361,
          262,  1766,     5,  4432,    69,   493,     0,     0,     0,
            0,     0],
       [    1,   701,   190,    19,     1,   425,    32,     3,  7359,
            4,  2153,  1253,     6,     3,  5231,     4,  1219,  1253,
           12,  3354],
       [  

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
texts_headings = dataset["Headline"]

article_headings = []

for text_heading in texts_headings:
  article_headings.append(sent_tokenize(text_heading))

data_headline = np.zeros((len(article_headings),MAX_SENTS_HEADING,MAX_SENT_LENGTH),dtype=int)

i=0
for article_heading in article_headings:
  j=0
  for sentence in article_heading:
    if j < 1:
      wordArr =  text_to_word_sequence(sentence,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“', lower=True, split=' ')
      k=0
      for word in wordArr:
        if k < 20:
          data_headline[i,j,k] = round(t.word_index[word])
          k+=1
      j+=1
  i+=1


In [22]:
data_headline[0,:,:]

array([[  720,   207,   344,  7086,   195,    36,  1340, 11389,    21,
          233,   687,     0,     0,     0,     0,     0,     0,     0,
            0,     0]])

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [23]:
stances = np.array(dataset["Stance"])
unique_stances = np.unique(stances)
print(unique_stances)

['agree' 'disagree' 'discuss' 'unrelated']


In [24]:
num_agree = len(dataset.loc[dataset['Stance'] == unique_stances[0]])
num_disagree = len(dataset.loc[dataset['Stance'] == unique_stances[1]])
num_discuss = len(dataset.loc[dataset['Stance'] == unique_stances[2]])
num_unrelated = len(dataset.loc[dataset['Stance'] == unique_stances[3]])

print("Headline agrees with article ",num_agree)
print("Headline disagrees with article ",num_disagree)
print("Article discusses what is in the headline ",num_discuss)
print("Article is unrelated to headline  ",num_unrelated)

Headline agrees with article  3678
Headline disagrees with article  840
Article discusses what is in the headline  8909
Article is unrelated to headline   36545


It is evident from the print statements above that the dataset is highly imbalanced. Before we balance the dataset, we will try to develop a model as is and see what the accuracy is. 

### Check 4:

The shape of data and labels should match the given below numbers.

### Shuffle the data

In [0]:
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_headline = data_headline[indices]
## shuffle the labels according to data

targets = pd.Series(stances)
one_hot = pd.get_dummies(targets, sparse = True)
one_hot_labels = np.asarray(one_hot)
labels = one_hot_labels

labels = labels[indices]

In [27]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, X_heading_train, X_heading_val, y_train,y_val = train_test_split(data,data_headline, labels, test_size = 0.20, random_state=1)


### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [29]:
print(X_train.shape)
print(X_heading_train.shape)
print(y_train.shape)

print(X_val.shape)
print(X_heading_val.shape)
print(y_val.shape)

(39977, 20, 20)
(39977, 1, 20)
(39977, 4)
(9995, 20, 20)
(9995, 1, 20)
(9995, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [30]:
# load the whole embedding into memory
embeddings_index = dict()
f = open(project_path+'glovefiles/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

vocab_size = len(t.word_index) + 1

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))


for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Dense, TimeDistributed, Activation,Bidirectional,Dropout, concatenate, LSTM
from tensorflow.keras.layers import Flatten, Permute, Input, Add
from tensorflow.keras.optimizers import Adam


### Model

In [32]:
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 100, input_length=(MAX_SENT_LENGTH,), weights=[embedding_matrix])(sentence_input)

bi_lstm = Bidirectional(LSTM(64, dropout=0.3, activation='tanh', recurrent_dropout=0.3, return_sequences=True))(embedded_sequences)
l_dense = Flatten()(TimeDistributed(Dense(100))(bi_lstm))
sentenceEncoder = Model(sentence_input, l_dense)

article_input = Input(shape=(MAX_SENTS,MAX_SENT_LENGTH,), dtype='int32')
article_encoder = TimeDistributed(sentenceEncoder)(article_input)
bi_lstm_article = Bidirectional(LSTM(64, dropout=0.3, activation='tanh', recurrent_dropout=0.3, return_sequences=True))(article_encoder)
article_dense_sent = Flatten()((TimeDistributed(Dense(100))(bi_lstm_article)))

heading_input = Input(shape=(MAX_SENTS_HEADING,MAX_SENT_LENGTH,), dtype='int32')
heading_encoder = TimeDistributed(sentenceEncoder)(heading_input)
bi_lstm_heading = LSTM(64, dropout=0.3, activation='tanh', recurrent_dropout=0.3, return_sequences=True)(heading_encoder)
heading_dense_sent = Flatten()((TimeDistributed(Dense(100))(bi_lstm_heading)))

article_output = concatenate([article_dense_sent, heading_dense_sent], name='concatenate_heading')

news_vector = Dense(100, activation='relu')(article_output)
preds = Dense(4, activation='softmax')(news_vector)
merged_model = Model([article_input, heading_input], [preds])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [33]:

# Concatenate the output of the two flattened layers

mergedOut = Dense(4, activation='softmax')(article_output)
merged_model = Model([article_input,heading_input], mergedOut)
merged_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 20, 20)]     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1, 20)]      0                                            
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, 20, 2000)     2803580     input_2[0][0]                    
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, 1, 2000)      2803580     input_3[0][0]                    
____________________________________________________________________________________________

### Compile and fit the model

In [34]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = "saved-model-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='max')

merged_model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

merged_model.fit([X_train,X_heading_train], y_train, epochs=5, validation_data=([X_val,X_heading_val], y_val),batch_size = 128,callbacks=[checkpoint])  

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 39977 samples, validate on 9995 samples
Epoch 1/5
39936/39977 [============================>.] - ETA: 0s - loss: 0.6897 - acc: 0.7530
Epoch 00001: saving model to saved-model-01-0.80.hdf5
39977/39977 [==============================] - 755s 19ms/sample - loss: 0.6897 - acc: 0.7529 - val_loss: 0.5731 - val_acc: 0.7997
Epoch 2/5
39936/39977 [============================>.] - ETA: 0s - loss: 0.5229 - acc: 0.8045
Epoch 00002: saving model to saved-model-02-0.82.hdf5
39977/39977 [==============================] - 739s 18ms/sample - loss: 0.5230 - acc: 0.8044 - val_loss: 0.4823 - val_acc: 0.8199
Epoch 3/5
39936/39977 [============================>.] - ETA: 0s - loss: 0.4600 - acc: 0.8256
Epoch 00003: saving model to saved-model-03-0.83.hdf5
39977/39977 [==============================] - 745s 19ms/sample - loss: 0.4598 - acc: 0.8257 - val_loss: 0.4535 - val_acc: 0.8323
Epoch 4/5
39936/39977 [

In [35]:
score = merged_model.evaluate([X_val,X_heading_val], y_val, verbose=0)
print("Accuracy: %.2f%%" % (score[1]*100))

Accuracy: 83.37%


The accuracy we have got good but the data is highly imbalanced as we saw earlier. Balancing the data and reattempting the model for classification with upsampled data will help in developing a more sustainable model. 